In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_HomePlanetNull_update.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9084_01,2.0,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8689,9194_01,1.0,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8690,9248_01,2.0,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8691,9257_01,2.0,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [4]:
df.groupby('Destination').count()
df.groupby('HomePlanet').count()
print('Cabin의 개수',len(df.groupby('Cabin').count()))

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
Destination,,,,,,,,,,,,,,,
0.0,1800,1800,1756,1750,1750,1750,1766,1757,1771,1763,1752,1761,1760,1757,1800
1.0,796,796,777,778,778,778,782,774,775,780,778,780,784,779,796
2.0,5915,5915,5765,5788,5788,5788,5787,5781,5787,5791,5778,5790,5781,5779,5915


,PassengerId,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
HomePlanet,,,,,,,,,,,,,,,
0.0,4714,4600,4616,4616,4616,4611,4626,4597,4608,4615,4598,4612,4603,4604,4714
1.0,2175,2115,2112,2112,2112,2138,2127,2133,2138,2129,2121,2130,2137,2128,2175
2.0,1804,1761,1766,1766,1766,1762,1761,1760,1766,1766,1766,1768,1765,1761,1804


Cabin의 개수 8


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [7]:
# df_cryosleep =  df_train중에서 CryoSleep의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_cryosleep = df[df['CryoSleep'].isnull()]

#df_train 중에서 CryoSleep의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df.dropna(subset=['CryoSleep'])
# df_train
df[df['CryoSleep'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
91,0099_02,0.0,NaN,6.0,12.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Thewis Connelson,1.0
97,0105_01,0.0,NaN,5.0,21.0,0.0,2.0,27.0,0.0,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,0.0
103,0110_02,1.0,NaN,1.0,5.0,0.0,2.0,40.0,0.0,0.0,331.0,0.0,0.0,1687.0,Aldeba Bootious,0.0
110,0115_01,2.0,NaN,5.0,24.0,0.0,2.0,26.0,0.0,0.0,0.0,0.0,0.0,NaN,Rohs Pead,1.0
150,0173_01,0.0,NaN,4.0,11.0,1.0,2.0,58.0,0.0,0.0,985.0,0.0,5.0,0.0,Hilip Grifford,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8466,9246_01,0.0,NaN,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8475,9259_01,0.0,NaN,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8486,9275_03,1.0,NaN,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8575,3896_01,1.0,NaN,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


In [8]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId','CryoSleep','Name'],axis=1),df_train01['CryoSleep'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(6780, 13)

(6780,)

(1696, 13)

(1696,)

In [9]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

0.9534198113207547

In [10]:
# df_01 은 df_train 중에서 CryoSleep의 값이 null인 경우의 데이터프레임
df_cryosleep_null = df_cryosleep.drop('CryoSleep',axis=1)
df_cryosleep_null

,PassengerId,HomePlanet,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
91,0099_02,0.0,6.0,12.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Thewis Connelson,1.0
97,0105_01,0.0,5.0,21.0,0.0,2.0,27.0,0.0,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,0.0
103,0110_02,1.0,1.0,5.0,0.0,2.0,40.0,0.0,0.0,331.0,0.0,0.0,1687.0,Aldeba Bootious,0.0
110,0115_01,2.0,5.0,24.0,0.0,2.0,26.0,0.0,0.0,0.0,0.0,0.0,NaN,Rohs Pead,1.0
150,0173_01,0.0,4.0,11.0,1.0,2.0,58.0,0.0,0.0,985.0,0.0,5.0,0.0,Hilip Grifford,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8466,9246_01,0.0,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8475,9259_01,0.0,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8486,9275_03,1.0,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8575,3896_01,1.0,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


In [11]:
# 훈련된 xgboost모델을 사용해서 df_cryosleep_null의 HomePlanet값을 예측
pred = model.predict(df_cryosleep_null.drop(['PassengerId','Name'],axis=1))
pred

array([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0])

In [12]:

# 예측된 값을 다시 df_cryosleep으로 삽입(null값 채우기)
df_cryosleep['CryoSleep'] = pred
df_cryosleep

C:\Users\shs03\AppData\Local\Temp\ipykernel_63284\2475673239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cryosleep['CryoSleep'] = pred


,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
91,0099_02,0.0,1,6.0,12.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Thewis Connelson,1.0
97,0105_01,0.0,0,5.0,21.0,0.0,2.0,27.0,0.0,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,0.0
103,0110_02,1.0,0,1.0,5.0,0.0,2.0,40.0,0.0,0.0,331.0,0.0,0.0,1687.0,Aldeba Bootious,0.0
110,0115_01,2.0,1,5.0,24.0,0.0,2.0,26.0,0.0,0.0,0.0,0.0,0.0,NaN,Rohs Pead,1.0
150,0173_01,0.0,0,4.0,11.0,1.0,2.0,58.0,0.0,0.0,985.0,0.0,5.0,0.0,Hilip Grifford,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8466,9246_01,0.0,1,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8475,9259_01,0.0,0,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8486,9275_03,1.0,0,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8575,3896_01,1.0,0,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


In [13]:

# null값을 채운 dataframe과 df_train01 합치기 
df_update = pd.concat([df_train01,df_cryosleep],axis=0)
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8466,9246_01,0.0,1.0,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8475,9259_01,0.0,0.0,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8486,9275_03,1.0,0.0,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8575,3896_01,1.0,0.0,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


In [14]:
df_update[df_update['CryoSleep'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [15]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_CryoSleepNull_update.csv', index=False)
